In [1]:
import cv2
import tkinter as tk
from tkinter import ttk
from ultralytics import YOLO
import pygame
from threading import Thread
from twilio.rest import Client
from datetime import datetime

pygame 2.6.1 (SDL 2.28.4, Python 3.11.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
account_sid = 'twilio_account_sid'
auth_token = 'twilio_auth_token'
twilio_phone_number = 'twilio_phone_number'
user_phone_number = 'your_phone_number'

In [3]:
client = Client(account_sid, auth_token)

In [4]:
model = YOLO(r"path to your knife model")

In [6]:
pygame.mixer.init()
pygame.mixer.music.load(r"path to siren.wav")

In [7]:
class WeaponDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Weapon Detection System")
        self.root.geometry("800x500")
        self.root.config(bg="#2E2E2E")

        self.video_capture = None
        self.is_running = False
        self.siren_playing = False

        header = tk.Label(self.root, text="Weapon Detection System",
                          font=("Helvetica", 24, "bold"), bg="#3B3B3B", fg="white")
        header.pack(fill=tk.X, pady=(10, 0))

        main_frame = ttk.Frame(self.root, padding=20)
        main_frame.pack(expand=True, fill=tk.BOTH)

        self.status_label = ttk.Label(main_frame, text="Status: Idle",
                                      font=("Arial", 18), foreground="#FF6347")
        self.status_label.pack(pady=10)

        button_frame = ttk.Frame(main_frame)
        button_frame.pack(pady=20)

        self.start_button = ttk.Button(button_frame, text="Start Detection",
                                       command=self.start_detection, style="TButton")
        self.start_button.grid(row=0, column=0, padx=10)

        self.stop_button = ttk.Button(button_frame, text="Stop Detection",
                                      command=self.stop_detection, style="TButton")
        self.stop_button.grid(row=0, column=1, padx=10)

        self.exit_button = ttk.Button(button_frame, text="Exit",
                                      command=self.exit_application, style="TButton")
        self.exit_button.grid(row=0, column=2, padx=10)

        style = ttk.Style()
        style.configure("TButton", font=("Arial", 14), padding=10)
        style.map("TButton", foreground=[('pressed', 'red'), ('active', 'blue')])

    def start_detection(self):
        """Start weapon detection."""
        if not self.is_running:
            self.is_running = True
            self.video_capture = cv2.VideoCapture(0)
            self.status_label.config(text="Status: Running...", foreground="green")
            Thread(target=self.detect_objects, daemon=True).start()

    def stop_detection(self):
        """Stop weapon detection."""
        self.is_running = False
        if self.video_capture:
            self.video_capture.release()
        self.status_label.config(text="Status: Stopped", foreground="orange")
        if self.siren_playing:
            pygame.mixer.music.stop()
            self.siren_playing = False

    def detect_objects(self):
        """Perform weapon detection using the camera."""
        while self.is_running:
            ret, frame = self.video_capture.read()
            if not ret:
                self.status_label.config(text="Status: Camera Error", foreground="red")
                break

            frame = cv2.flip(frame, 1)
            results = model.predict(source=frame, conf=0.6, show=False)
            annotated_frame = results[0].plot()

            knife_detected = any("knife" in result.names[int(cls)].lower() 
                                 for result in results for cls in result.boxes.cls)

            if knife_detected:
                self.status_label.config(text="Status: Knife Detected!", foreground="red")
                self.trigger_alert()
            else:
                self.status_label.config(text="Status: No Knife Detected", foreground="green")

            cv2.imshow("Weapon Detection", annotated_frame)

            if cv2.waitKey(1) & 0xFF == 27:
                break

        self.stop_detection()
        cv2.destroyAllWindows()

    def trigger_alert(self):
        """Trigger siren and send SMS alert."""
        if not self.siren_playing:
            pygame.mixer.music.play(-1)
            self.siren_playing = True

        # Get the current time
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Send SMS alert
        try:
            message_body = f"Warning! A knife has been detected at {current_time}."
            client.messages.create(
                body=message_body,
                from_=twilio_phone_number,
                to=user_phone_number
            )
            print("SMS alert sent successfully!")
        except Exception as e:
            print(f"Failed to send SMS alert: {e}")

    def exit_application(self):
        """Exit the application."""
        self.stop_detection()
        cv2.destroyAllWindows()
        pygame.mixer.quit()
        self.root.destroy()

In [8]:
if __name__ == "__main__":
    root = tk.Tk()
    app = WeaponDetectionApp(root)
    root.mainloop()


0: 480x640 (no detections), 119.8ms
Speed: 7.8ms preprocess, 119.8ms inference, 43.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 13.7ms
Speed: 3.1ms preprocess, 13.7ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 21.1ms
Speed: 3.4ms preprocess, 21.1ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 11.3ms
Speed: 2.7ms preprocess, 11.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 17.9ms
Speed: 5.3ms preprocess, 17.9ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 18.4ms
Speed: 3.1ms preprocess, 18.4ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 22.5ms
Speed: 3.3ms preprocess, 22.5ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 12.8ms
Speed: 4.1ms preprocess, 12.8m